# Familial DistilBERT Model Using Merged Data Experiment 1.1

In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [2]:
merged_familial_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_Familial_sentence_level_batch_1_jaccard.csv", encoding='utf-8')

# Shuffle the merged dataset
merged_familial_df = shuffle(merged_familial_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_familial_df, test_size=0.1, random_state=18, stratify=merged_familial_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

## 2. Experimental Design

In [3]:
MAXLEN = 150

In [4]:
X = training_df['sentence']
y = training_df['label']

In [5]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [6]:
model_name = 'distilbert-base-uncased'

In [7]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 43
	99percentile : 55


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 37
	99percentile : 54


In [8]:
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [9]:
distillbert_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
173/173 [==============================] - 47s 205ms/step - loss: 0.3424 - accuracy: 0.8871 - val_loss: 0.2458 - val_accuracy: 0.8692
Epoch 2/5
173/173 [==============================] - 28s 161ms/step - loss: 0.2102 - accuracy: 0.8967 - val_loss: 0.2385 - val_accuracy: 0.8577
Epoch 3/5
173/173 [==============================] - 28s 162ms/step - loss: 0.1733 - accuracy: 0.8986 - val_loss: 0.2189 - val_accuracy: 0.8769
Epoch 4/5
173/173 [==============================] - 29s 164ms/step - loss: 0.1576 - accuracy: 0.9208 - val_loss: 0.2313 - val_accuracy: 0.8731
Epoch 5/5
173/173 [==============================] - 29s 165ms/step - loss: 0.1356 - accuracy: 0.9199 - val_loss: 0.2659 - val_accuracy: 0.8615


In [10]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

9/9 [==============================] - 5s 306ms/step
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       232
           1       0.38      0.43      0.40        28

    accuracy                           0.86       260
   macro avg       0.65      0.67      0.66       260
weighted avg       0.87      0.86      0.87       260



array([[212,  20],
       [ 16,  12]])

In [11]:
distillbert_learner.model.summary()

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [13]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [14]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [15]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [16]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 120, False Positive: 8, False Negative: 8, True Positive: 8


In [17]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       128
           1       0.50      0.50      0.50        16

    accuracy                           0.89       144
   macro avg       0.72      0.72      0.72       144
weighted avg       0.89      0.89      0.89       144
 



In [18]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [19]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.71875
